In [151]:
import pandas as pd

In [152]:
# === Step 1: Load and prepare the option data ===

def load_and_prepare_data_all_days(option_data_path, vol_data_path):
    # Load the full options and volatility datasets
    option_data = pd.read_csv(option_data_path)
    option_data['date'] = pd.to_datetime(option_data['date'])
    option_data['exdate'] = pd.to_datetime(option_data['exdate'])

    vol_data = pd.read_csv(vol_data_path)
    vol_data['Date'] = pd.to_datetime(vol_data['Date'])

    # Get unique sorted analysis dates
    all_dates = sorted(option_data['date'].unique())

    results = []  # store per-day outputs

    for analysis_date in all_dates:
        # Filter option data for that date
        daily_data = option_data[option_data['date'] == analysis_date].copy()
        daily_data = daily_data.dropna(subset=['delta', 'gamma'])
        daily_data.reset_index(drop=True, inplace=True)

        # === Volatility check ===
        vol_row = vol_data[vol_data['Date'] == analysis_date]

        if vol_row.empty:
            print(f"No vol data for {analysis_date.date()} – skipping.")
            continue

        realized_vol = vol_row['Realized_Volatility'].values[0]
        vol_target = 0.20

        # Skip hedge if volatility is below target
        if realized_vol < vol_target:
            print(f"Vol < target on {analysis_date.date()} ({realized_vol:.2%}) – skipping hedge.")
            continue

        print(f"✅ Processing {analysis_date.date()}...")

        # Only append days we are actually hedging
        results.append((analysis_date, daily_data))

    return results


In [153]:
# === Step 2: Define portfolio ===

def define_portfolio(daily_data, analysis_date, portfolio_value=10_000_000):

    """
    Estimate QQQ price from ATM options, calculate number of shares, and portfolio delta.
    """
    # Step 2.1: Fix strike prices if needed
    if daily_data['strike_price'].max() > 1000:
        daily_data['strike_price'] = daily_data['strike_price'] / 100

    # Step 2.2: Find ATM options (delta between 0.4 and 0.6)
    atm_options = daily_data[(daily_data['delta'] > 0.4) & (daily_data['delta'] < 0.6)]

    if atm_options.empty:
        raise ValueError("No ATM options found – cannot estimate QQQ price for this day.")

    # Step 2.3: Estimate QQQ price using mean strike of ATM options
    estimated_qqq_price = atm_options['strike_price'].mean()

    # Step 2.4: Calculate number of shares
    num_shares = float(portfolio_value) / estimated_qqq_price

    # Step 2.5: Portfolio delta (assume 1 delta per QQQ share)
    portfolio_delta = num_shares * 1

    # Return clean scalar values, not Pandas objects
    return {
        'estimated_qqq_price': float(estimated_qqq_price),
        'num_shares': float(num_shares),
        'portfolio_delta': float(portfolio_delta),
        'portfolio_value': float(portfolio_value),
        'analysis_date': analysis_date  # ✅ Add this line
    }



In [154]:
# === Step 3: Select Hedging Options ===

def select_hedging_options(daily_data):
    """
    Select options for delta hedging and gamma hedging with cost tracking.
    Returns two DataFrames: put_options and high_gamma_options.
    """

    # Step 3.1: Select put options for delta hedging (delta < 0)
    put_options = daily_data[(daily_data['cp_flag'] == 'P') & (daily_data['delta'] < 0)].copy()

    # Add cost tracking (mid-price and spread)
    put_options['mid_price'] = (put_options['best_bid'] + put_options['best_offer']) / 2
    put_options['spread'] = put_options['best_offer'] - put_options['best_bid']

    # Optional: filter out puts with huge bid-ask spreads
    put_options = put_options[put_options['spread'] < 5]

    # Step 3.2: Select high-gamma options (top 10%)
    gamma_threshold = daily_data['gamma'].quantile(0.90)
    high_gamma_options = daily_data[daily_data['gamma'] > gamma_threshold].copy()

    high_gamma_options['mid_price'] = (high_gamma_options['best_bid'] + high_gamma_options['best_offer']) / 2
    high_gamma_options['spread'] = high_gamma_options['best_offer'] - high_gamma_options['best_bid']

    # Optional: prioritize higher gamma
    high_gamma_options = high_gamma_options.sort_values(by='gamma', ascending=False)

    return put_options, high_gamma_options

In [155]:
# === Step 4: Calculate Hedge Ratios ===
def calculate_hedge_ratios(portfolio, option_row, gamma_per_share=0.002): # adjust gammer per share later, this is a placeholder
    """
    Calculate how many option contracts are needed to hedge delta and gamma risk.
    
    Parameters:
    - portfolio: dict with info like portfolio delta and number of shares
    - option_row: one row of option data (with delta/gamma info)
    - gamma_per_share: estimated gamma per QQQ share (default = 0.002)
    
    Returns:
    - delta_hedge_contracts: how many contracts needed to hedge portfolio delta
    - gamma_hedge_contracts: how many contracts needed to hedge portfolio gamma
    """

    # Get portfolio delta from the dictionary (already calculated earlier)
    portfolio_delta = portfolio['portfolio_delta']

    # Estimate portfolio gamma = number of shares * gamma per share
    portfolio_gamma = portfolio['num_shares'] * gamma_per_share

    # Extract option Greeks from the row (each row is one option)
    delta = option_row['delta']
    gamma = option_row['gamma']

    # Delta Hedge:
    # Divide total portfolio delta by the option's delta
    # We take the absolute value of option delta since puts are negative
    delta_hedge_contracts = portfolio_delta / abs(delta) if delta != 0 else float('inf')

    # Gamma Hedge:
    # Divide total portfolio gamma by the option's gamma
    gamma_hedge_contracts = portfolio_gamma / gamma if gamma != 0 else float('inf')

    return delta_hedge_contracts, gamma_hedge_contracts



In [156]:
# === Step 5: Store & Output Best Hedge Plan ===

def build_hedge_plan(portfolio, put_options, high_gamma_options, analysis_date):
    """
    Build a hedge plan using the *best* put option for delta hedging
    and the *best* high-gamma option for gamma hedging — one of each.

    Parameters:
    - portfolio: Dictionary containing portfolio value, number of shares, delta, etc.
    - put_options: DataFrame of filtered put options (for delta hedge)
    - high_gamma_options: DataFrame of filtered high-gamma options (for gamma hedge)
    - analysis_date: Timestamp of current trading day (added as a column)

    Returns:
    - hedge_df: DataFrame summarizing the hedge plan (1 row per hedge type)
    """

    # === Initialize ===
    hedge_plan = []  # List to collect rows of hedge information
    put_options = put_options.reset_index(drop=True)
    high_gamma_options = high_gamma_options.reset_index(drop=True)

    # === Delta Hedge Section ===
    if not put_options.empty:
        # Step 1: Score each option: hedging power / cost
        put_options = put_options.copy()
        put_options['score'] = abs(put_options['delta']) / (put_options['mid_price'] + put_options['spread'])

        # Step 2: Pick best delta hedge option (highest score)
        best_put = put_options.sort_values('score', ascending=False).iloc[0]

        # Step 3: Calculate # contracts needed for delta hedge
        delta_contracts, _ = calculate_hedge_ratios(portfolio, best_put)

        # Step 4: Store result
        hedge_plan.append({
            'date': analysis_date,                         # track day
            'hedge_type': 'delta',                         # delta hedge
            'cp_flag': best_put['cp_flag'],                # C = Call, P = Put
            'strike_price': best_put['strike_price'],      # option strike
            'delta': best_put['delta'],                    # option delta
            'gamma': best_put['gamma'],                    # option gamma
            'mid_price': best_put['mid_price'],            # estimated mid price
            'spread': best_put['spread'],                  # bid-ask spread
            'contracts_needed': delta_contracts            # # needed to hedge delta
        })

    # === Gamma Hedge Section ===
    if not high_gamma_options.empty:
        # Step 1: Score each option: gamma power / cost
        high_gamma_options = high_gamma_options.copy()
        high_gamma_options['score'] = abs(high_gamma_options['gamma']) / (high_gamma_options['mid_price'] + high_gamma_options['spread'])

        # Step 2: Pick best gamma hedge option (highest score)
        best_gamma = high_gamma_options.sort_values('score', ascending=False).iloc[0]

        # Step 3: Calculate # contracts needed for gamma hedge
        gamma_contracts = calculate_hedge_ratios(portfolio, best_gamma)

        # Step 4: Store result
        hedge_plan.append({
            'date': analysis_date,                         # track day
            'hedge_type': 'gamma',                         # gamma hedge
            'cp_flag': best_gamma['cp_flag'],              # C = Call, P = Put
            'strike_price': best_gamma['strike_price'],    # option strike
            'delta': best_gamma['delta'],                  # option delta
            'gamma': best_gamma['gamma'],                  # option gamma
            'mid_price': best_gamma['mid_price'],          # estimated mid price
            'spread': best_gamma['spread'],                # bid-ask spread
            'contracts_needed': gamma_contracts[0]          # # needed to hedge gamma
        })

    # === Convert list to DataFrame ===
    hedge_df = pd.DataFrame(hedge_plan)
    return hedge_df



In [157]:
# === Run Hedging Strategy Across All Dates ===
# === Run Hedging Strategy Across All Dates ===

# Load and prepare data
results = load_and_prepare_data_all_days(
    option_data_path="Data/qqq_option_data_2018_2023.csv",
    vol_data_path="Data/dataset1.csv"
)

# List to hold each day's hedge plan
full_hedge_plans = []

# Run Steps 2–5 for each day
for analysis_date, daily_data in results:
    print(f"\n––– Running hedge plan for {analysis_date.date()} –––")

    # Step 2: Define portfolio
    portfolio = define_portfolio(daily_data, analysis_date)

    # Step 3: Select hedging options
    put_options, high_gamma_options = select_hedging_options(daily_data)

    # Step 4: Store and save the daily hedge plan (Step 4 is called inside this)
    #daily_plan = build_hedge_plan(portfolio, put_options, high_gamma_options)
    daily_plan = build_hedge_plan(portfolio, put_options, high_gamma_options, analysis_date)


    # Append to full results
    full_hedge_plans.append(daily_plan)

# Combine all days into one final DataFrame
final_hedge_df = pd.concat(full_hedge_plans, ignore_index=True)

# Export final result to CSV
final_hedge_df.to_csv("final_hedge_plan.csv", index=False)

print("\n✅ Hedging strategy complete! File saved as 'final_hedge_plan.csv'")


No vol data for 2018-01-02 – skipping.
No vol data for 2018-01-03 – skipping.
No vol data for 2018-01-04 – skipping.
No vol data for 2018-01-05 – skipping.
No vol data for 2018-01-08 – skipping.
No vol data for 2018-01-09 – skipping.
No vol data for 2018-01-10 – skipping.
No vol data for 2018-01-11 – skipping.
No vol data for 2018-01-12 – skipping.
No vol data for 2018-01-16 – skipping.
No vol data for 2018-01-17 – skipping.
No vol data for 2018-01-18 – skipping.
No vol data for 2018-01-19 – skipping.
No vol data for 2018-01-22 – skipping.
No vol data for 2018-01-23 – skipping.
No vol data for 2018-01-24 – skipping.
No vol data for 2018-01-25 – skipping.
No vol data for 2018-01-26 – skipping.
No vol data for 2018-01-29 – skipping.
No vol data for 2018-01-30 – skipping.
No vol data for 2018-01-31 – skipping.
No vol data for 2018-02-01 – skipping.
No vol data for 2018-02-02 – skipping.
No vol data for 2018-02-05 – skipping.
No vol data for 2018-02-06 – skipping.
No vol data for 2018-02-0

> Each row in the CSV represents a single option contract selected to hedge the portfolio on a specific day. It includes the date the hedge was executed, whether the option was used to hedge delta or gamma risk, the type of option (call or put), the strike price, and key option characteristics like delta, gamma, mid-market price, and bid-ask spread. The final column, "contracts_needed," tells you how many contracts of that specific option are required to neutralize either the delta or gamma risk of the portfolio for that day.

> Multiple rows on the same date mean that several candidate options were initially considered—usually the top five—for better precision and flexibility in constructing the hedge. To pick the best one, we created a simple scoring method: we divided the number of contracts needed by the option’s bid-ask spread. This gave us a “hedge score” that penalizes options requiring a lot of contracts or having a large spread (which can lead to high slippage). The lower the score, the cheaper and more efficient the hedge.

> We explored other methods too, like selecting the option with the smallest spread or the one with the highest gamma exposure. But ultimately, we went with the hedge score method because it struck a balance between minimizing cost and reducing execution risk. Only the best-scoring option for each hedge type (delta and gamma) was kept in the final output.


In [158]:
final_csv  = pd.read_csv("/Users/nzaramakouadio/Documents/Duke Classes/Spring 2025/RISK/583_RiskManagement_Final_Project/final_hedge_plan.csv")

In [159]:
final_csv.head()

,date,hedge_type,cp_flag,strike_price,delta,gamma,mid_price,spread,contracts_needed
0,2018-02-22,delta,P,1525.0,-0.138812,0.018333,0.880,0.02,43660.534108
1,2018-02-22,gamma,C,1685.0,0.327754,0.047452,1.605,0.07,18491.325996
2,2018-02-23,delta,P,1585.0,-0.158830,0.024259,0.855,0.03,37461.561628
3,2018-02-23,gamma,C,1720.0,0.269699,0.054145,0.995,0.07,22061.705210
4,2018-02-26,delta,P,1615.0,-0.162650,0.026726,0.810,0.02,36090.782329
